In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
!pip install xgboost
import xgboost as xgb

In [ ]:
df_train= pd.read_csv('ccfrd train.csv')
df_test= pd.read_csv('ccfrd test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.drop(columns=['Unnamed: 0'])
df_test.drop(columns=['Unnamed: 0'])

In [ ]:
df_train.drop(columns=['Unnamed: 0', 'first', 'last', 'street', 'cc_num', 'trans_num'])
df_test.drop(columns=['Unnamed: 0', 'first', 'last', 'street', 'cc_num', 'trans_num'])

In [ ]:
df_train['trans_date_trans_time'] = pd.to_datetime(df_train['trans_date_trans_time'])
df_test['trans_date_trans_time'] = pd.to_datetime(df_test['trans_date_trans_time'])

In [ ]:
df_train['dob'] = pd.to_datetime(df_train['dob'])
df_test['dob'] = pd.to_datetime(df_test['dob'])

In [ ]:
df_train['age'] = (df_train['trans_date_trans_time'] - df_train['dob']).dt.days // 365
df_test['age'] = (df_test['trans_date_trans_time'] - df_test['dob']).dt.days // 365

In [ ]:
for df in [df_train, df_test]:
    df['hour'] = df['trans_date_trans_time'].dt.hour
    df['day_of_week'] = df['trans_date_trans_time'].dt.dayofweek
    df['day'] = df['trans_date_trans_time'].dt.day
    df['month'] = df['trans_date_trans_time'].dt.month


In [ ]:
df_train.drop(['dob', 'trans_date_trans_time'], axis=1, inplace=True)
df_test.drop(['dob', 'trans_date_trans_time'], axis=1, inplace=True)

In [ ]:
df_train['gender'] = df_train['gender'].map({'F': 0, 'M': 1})
df_test['gender'] = df_test['gender'].map({'F': 0, 'M': 1})


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
sns.countplot(x='is_fraud', data=df_train)
plt.title('Class Distribution in Training Set')
plt.show()


In [ ]:
corr_matrix = df_train.corr(numeric_only=True)


In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
df_combined = pd.concat([df_train, df_test], axis=0)


In [ ]:
categorical_cols = ['merchant', 'category', 'city', 'state', 'job']
for col in categorical_cols:
    le = LabelEncoder()
    df_combined[col] = le.fit_transform(df_combined[col])


In [ ]:
df_train_encoded = df_combined.iloc[:len(df_train)]
df_test_encoded = df_combined.iloc[len(df_train):]


In [ ]:
X = df_train_encoded.drop('is_fraud', axis=1)
y = df_train_encoded['is_fraud']


In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


In [ ]:
# Initialize XGBoost classifier
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

print("Classification Report:")
print(classification_report(y_val, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

print(f"ROC AUC Score: {roc_auc_score(y_val, y_proba):.4f}")


In [ ]:
y_pred = model.predict(X_val)
y_proba = model.predict_proba(X_val)[:, 1]


In [ ]:
# Evaluation metrics

print("Classification Report:")
print(classification_report(y_val, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))
print(f"ROC AUC Score: {roc_auc_score(y_val, y_proba):.4f}")


In [ ]:
# Prepare test features
X_test = df_test_encoded.drop('is_fraud', axis=1)

# Predict probabilities
test_proba = model.predict_proba(X_test)[:, 1]

# Predict classes
test_pred = model.predict(X_test)


In [ ]:
df_test['predicted_proba'] = test_proba
df_test['predicted_label'] = test_pred

# Display sample predictions
df_test[['predicted_proba', 'predicted_label']].head()
